In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

2025-04-17 17:14:51.866297: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 17:14:51.946984: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-17 17:14:51.947039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-17 17:14:51.948943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 17:14:51.960646: I tensorflow/core/platform/cpu_feature_guar

GPU is available


In [2]:
# Read from CSV instead of pickle
dataset = pd.read_csv('cleaned_profiles.csv')
dataset

,Intro,Full Name,Workplace,Location,Connections,Photo,Followers,About,Experiences,Number of Experiences,...,Number of Scores,Languages,Number of Languages,Organizations,Number of Organizations,Interests,Number of Interests,Activities,Number of Activities,Label
0,"{'Full Name': 'chenxia (polly) Pei', 'Workplac...",chenxia (polly) Pei,Jiangsu Junyao mainly offer services to cement...,"Wuxi, Jiangsu, China",500,No,717,NaN,"{'0': {'Role': 'Sales', 'Workplace': 'Jiangsu ...",2,...,0,{},0,{},0,"{'0': {'Interest': 'Trina Solar', 'ID': '69648...",4,{'chenxia-pei-80177594': {'Full Name': 'chenxi...,1,1
1,"{'Full Name': 'NEHA CHANDOK', 'Workplace': 'So...",NEHA CHANDOK,Software Analyst,"Noida, Uttar Pradesh, India",500,No,1340,NaN,"{'0': {'Role': 'Software Analyst', 'Workplace'...",1,...,0,"{'Language 0': 'Bangla', 'Language 1': 'Englis...",3,{},0,"{'0': {'Interest': 'Mohamed El-Erian', 'ID': '...",8,{},0,1
2,"{'Full Name': 'Mounika Mungamuri', 'Workplace'...",Mounika Mungamuri,Senior Consultant at Infosys,"Hyderabad, Telangana, India",7,Yes,7,NaN,"{'0': {'Role': 'Senior Consultant', 'Workplace...",2,...,0,{},0,{},0,{},0,{},0,1
3,"{'Full Name': 'Katarina Djuric', 'Workplace': ...",Katarina Djuric,--,"Belgrade, Serbia",0,Yes,0,NaN,"{'0': {'Role': 'Instructor', 'Workplace': 'GE'...",1,...,0,{},0,{},0,"{'0': {'Interest': 'GE', 'ID': '1015', 'Catego...",1,{},0,1
4,"{'Full Name': 'Rachel Lally', 'Workplace': '--...",Rachel Lally,--,"Dublin, County Dublin, Ireland",61,Yes,61,NaN,"{'0': {'Role': 'Bartender', 'Workplace': ""O'Su...",1,...,1,{},0,{},0,"{'0': {'Interest': 'Richard Branson', 'ID': 'r...",3,"{'garyltravis': {'Full Name': 'Gary Travis', '...",6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,"{'Full Name': 'Emily Williams', 'Workplace': ""...",Emily Williams,Market Research Analyst at L'Oreal,"New York City, New York",106,No,717,{},Market Research Analyst Elizabeth Arden Jan 20...,3,...,0,{},0,{},0,{},3,{},4,11
3596,"{'Full Name': 'Sarah Johnson', 'Workplace': 'D...",Sarah Johnson,Director of Marketing Strategy at Acme Inc.,"San Francisco, California",102,No,6,{},Director of Marketing Acme Inc. January 2018-P...,5,...,0,{},0,{},0,{},5,{},3,11
3597,"{'Full Name': 'Sarah Johnson', 'Workplace': 'M...",Sarah Johnson,Manager at Acme Inc.,"New York City, New York",435,No,10,A results-driven and experienced professional ...,{},0,...,0,{},0,{},0,{},6,{},0,11
3598,"{'Full Name': 'Emma Williams', 'Workplace': 'M...",Emma Williams,Manager and Director at ABC Inc.,"Seattle, Washington",280,No,34,With over 10 years of experience in operations...,Operations Manager ABC Inc. January 2015- Pres...,1,...,0,English Spanish,2,{},0,{},4,{},4,11


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import brier_score_loss
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

def get_classifier(classifier_name, random_state=42):
    """Return classifier instance based on name"""
    if classifier_name == 'RF':
        return RandomForestClassifier(random_state=random_state)
    elif classifier_name == 'LR':
        return LogisticRegression(random_state=random_state, max_iter=1000)
    elif classifier_name == 'SVC':
        # Use probability=True to enable predict_proba
        return SVC(random_state=random_state, probability=True)
    elif classifier_name == 'KNN':
        return KNeighborsClassifier()
    elif classifier_name == 'XGB':
        return XGBClassifier(random_state=random_state)
    elif classifier_name == 'CatB':
        return CatBoostClassifier(random_state=random_state, verbose=False)

def calculate_metrics(y_true, y_pred):
    """Calculate F1, FAR, and FRR"""
    f1 = f1_score(y_true, y_pred, average='weighted')
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    far = fp / (fp + tn)
    frr = fn / (fn + tp)
    return {'f1_score': f1, 'far': far, 'frr': frr}
        
def plot_calibration_curves(classifiers, X_train, y_train, X_test, y_test, model_name, n_bins=10):
    """
    Plot calibration curves for each classifier
    
    Args:
        classifiers: Dictionary of classifier names and their instances
        X_train, y_train: Training data
        X_test, y_test: Test data
        model_name: Name of the embedding model
        n_bins: Number of bins for calibration curve
    """
    # Change figure size to match the comparative plot
    plt.figure(figsize=(5, 3.5), dpi=300)  # Smaller size, higher resolution
    
    # Use a single subplot instead of grid
    ax1 = plt.subplot()
    
    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    
    for clf_name, clf in classifiers.items():
        # Fit the classifier
        clf.fit(X_train, y_train)
        
        if hasattr(clf, "predict_proba"):
            prob_pos = clf.predict_proba(X_test)[:, 1]
        else:
            # Apply CalibratedClassifierCV for models without predict_proba
            calibrated_clf = CalibratedClassifierCV(clf, cv=5)
            calibrated_clf.fit(X_train, y_train)
            prob_pos = calibrated_clf.predict_proba(X_test)[:, 1]
        
        # Calculate calibration curve
        fraction_of_positives, mean_predicted_value = calibration_curve(
            y_test, prob_pos, n_bins=n_bins
        )
        
        # Calculate Brier score
        brier_score = brier_score_loss(y_test, prob_pos)
        
        # Plot calibration curve with smaller markers
        ax1.plot(
            mean_predicted_value,
            fraction_of_positives,
            "s-",
            markersize=2,  # Smaller markers
            linewidth=1,   # Thinner lines
            label=f"{clf_name} ({brier_score:.3f})"  # Shorter label text
        )
    
    # Adjust font sizes to match comparative plot
    ax1.set_title(f'Calibration Curves - {model_name}', fontsize=12)  # Shorter title
    ax1.set_ylabel("Fraction of positives", fontsize=12)
    ax1.set_xlabel("Mean predicted probability", fontsize=12)
    ax1.legend(loc="lower right", fontsize=9)  # Smaller legend font
    
    # Adjust tick sizes to match comparative plot
    ax1.tick_params(axis='both', which='major', labelsize=8)
    
    # Adjust layout to ensure everything fits
    plt.tight_layout(pad=0.3)
    
    # Save with higher resolution, ensure tight bounding box
    plt.savefig(f'calibration_curve_{model_name}.png', dpi=300, bbox_inches='tight')
    plt.close()

def evaluate_with_calibration():
    """Evaluate classifiers with calibration curves for multiple embedding models"""
    # List of embedding models to evaluate
    embedding_models = ['roberta', 'deberta', 'modernbert', 'flair']
    
    classifiers = {
        'RF': get_classifier('RF'),
        'LR': get_classifier('LR'),
        'SVC': get_classifier('SVC'),
        'KNN': get_classifier('KNN'),
        'XGB': get_classifier('XGB'),
        'CatB': get_classifier('CatB')
    }
    
    all_results = {}
    
    # Process each embedding model
    for model in embedding_models:
        print(f"\n{'='*20} {model.upper()} EMBEDDINGS {'='*20}")
        
        # Load embeddings (adjust path according to your file structure)
        emb_path = f'embeddings_output/{model}_embeddings.csv'
        try:
            emb = pd.read_csv(emb_path)
            print(f"Loaded embeddings from {emb_path}")
        except FileNotFoundError:
            print(f"Warning: Could not find {emb_path}. Using a placeholder path.")
            # This is a placeholder - replace with actual path if needed
            emb = pd.read_csv(f'embeddings_output/{model}_PCA_150_components.csv')
        
        # Get masks for different profile types
        # Assuming dataset is already loaded or adjust as needed
        genuine_mask = dataset['Label'] == 0
        fake_manual_mask = dataset['Label'] == 1
        
        # Get genuine and fake profiles
        X_genuine = emb[genuine_mask]
        X_fake = emb[fake_manual_mask]
        
        # Split the data 70-30
        X_train_genuine, X_test_genuine = train_test_split(X_genuine, test_size=0.3, random_state=42)
        X_train_fake, X_test_fake = train_test_split(X_fake, test_size=0.3, random_state=42)
        
        # Combine training and testing data
        X_train = pd.concat([X_train_genuine, X_train_fake])
        y_train = np.array([0]*len(X_train_genuine) + [1]*len(X_train_fake))
        
        X_test = pd.concat([X_test_genuine, X_test_fake])
        y_test = np.array([0]*len(X_test_genuine) + [1]*len(X_test_fake))
        
        # Print dataset composition
        print("\nDataset Split Details:")
        print("-" * 50)
        print(f"Training: {len(X_train)} samples ({len(X_train_genuine)} genuine, {len(X_train_fake)} fake)")
        print(f"Testing: {len(X_test)} samples ({len(X_test_genuine)} genuine, {len(X_test_fake)} fake)")
        
        # Generate calibration curves
        plot_calibration_curves(classifiers, X_train, y_train, X_test, y_test, model)
        
        # Evaluate each classifier
        model_results = {}
        for clf_name, clf in classifiers.items():
            # Fit classifier
            clf.fit(X_train, y_train)
            
            # Regular predictions
            y_pred = clf.predict(X_test)
            metrics = calculate_metrics(y_test, y_pred)
            
            # Calibrated predictions - using sigmoid calibration
            calibrated_clf = CalibratedClassifierCV(clf, cv='prefit', method='sigmoid')
            calibrated_clf.fit(X_test, y_test)  # Fit on test data for demonstration
            y_pred_calibrated = calibrated_clf.predict(X_test)
            metrics_calibrated = calculate_metrics(y_test, y_pred_calibrated)
            
            # Store results
            model_results[clf_name] = {
                'original': metrics,
                'calibrated': metrics_calibrated
            }
            
            # Print results
            print(f"\n{clf_name}:")
            print(f"  Original  - F1: {metrics['f1_score']:.4f}, FAR: {metrics['far']:.4f}, FRR: {metrics['frr']:.4f}")
            print(f"  Calibrated - F1: {metrics_calibrated['f1_score']:.4f}, FAR: {metrics_calibrated['far']:.4f}, FRR: {metrics_calibrated['frr']:.4f}")
        
        all_results[model] = model_results
    
    return all_results

if __name__ == "__main__":
    # Assuming dataset is already loaded or add code to load it
    # dataset = pd.read_csv('your_dataset.csv')
    
    # Evaluate with calibration curves
    results = evaluate_with_calibration()
    
    # Optional: Save results to CSV for further analysis
    results_df = []
    for model, model_results in results.items():
        for clf_name, metrics in model_results.items():
            results_df.append({
                'embedding_model': model,
                'classifier': clf_name,
                'original_f1': metrics['original']['f1_score'],
                'original_far': metrics['original']['far'],
                'original_frr': metrics['original']['frr'],
                'calibrated_f1': metrics['calibrated']['f1_score'],
                'calibrated_far': metrics['calibrated']['far'],
                'calibrated_frr': metrics['calibrated']['frr']
            })
    
    pd.DataFrame(results_df).to_csv('calibration_results.csv', index=False)
    print("\nResults saved to 'calibration_results.csv'")